In [1]:
import numpy as np
from scipy.spatial import ConvexHull as ch
from scipy.spatial import Delaunay

from coxeter.families import (
    ArchimedeanFamily as archfam,
)
from coxeter.families import (
    JohnsonFamily as johnfam,
)
from coxeter.families import (
    PlatonicFamily as platfam,
)
from coxeter.shapes import ConvexPolyhedron

In [ ]:
# DISCUSSION : find coplanar is pretty close to optimal. What I have written is an O(N) algorithm, so even though it
# does not use numpy it is very fast.

In [2]:
cube = platfam.get_shape("Cube")
tetr = platfam.get_shape("Tetrahedron")
mbri = johnfam.get_shape("Metabidiminished Rhombicosidodecahedron")
icosi = archfam.get_shape("Icosidodecahedron")
poly = mbri

In [3]:
hull = ch(poly.vertices, "")
normals = hull.equations[:, :3]
boolean_coplanars = np.all(normals[:, None] == normals, axis=-1)
boolean_coplanars

array([[ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False,  True,  True],
       [False, False, False, ..., False,  True,  True]])

In [4]:
rows = np.arange(0, hull.nsimplex)
coplanarray = np.tile(np.arange(0, hull.nsimplex), (hull.nsimplex, 1))
# coplanarray

In [5]:
%%timeit -r 10 -n 1000
boolean_coplanars = np.all(normals[:, None] == normals, axis=-1)
rows = np.arange(0, hull.nsimplex)
coplanar_simplices = {tuple(rows[boolean_coplanars[i]]) for i in range(hull.nsimplex)}
# Do we need to guarantee that the order of coplanars is the same
coplanar_simplices

271 µs ± 14.5 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [6]:
boolean_coplanars = np.all(normals[:, None] == normals, axis=-1)
simplex_indices = np.arange(0, hull.nsimplex)
coplanar_simplices = [
    simplex_indices[boolean_coplanars[i]] for i in range(hull.nsimplex)
]
# We need to guarantee that the ORDER of coplanar simplices is identical to the original order
len(coplanar_simplices), coplanar_simplices

(96,
 [array([0]),
  array([1]),
  array([2]),
  array([3]),
  array([4]),
  array([5]),
  array([6]),
  array([7]),
  array([8]),
  array([9]),
  array([10, 11]),
  array([10, 11]),
  array([12, 13]),
  array([12, 13]),
  array([14, 15, 16]),
  array([14, 15, 16]),
  array([14, 15, 16]),
  array([17, 18]),
  array([17, 18]),
  array([19, 20]),
  array([19, 20]),
  array([21, 22]),
  array([21, 22]),
  array([23, 24]),
  array([23, 24]),
  array([25, 26]),
  array([25, 26]),
  array([27, 28, 29]),
  array([27, 28, 29]),
  array([27, 28, 29]),
  array([30, 31]),
  array([30, 31]),
  array([32, 33, 34]),
  array([32, 33, 34]),
  array([32, 33, 34]),
  array([35, 36]),
  array([35, 36]),
  array([37, 38, 39]),
  array([37, 38, 39]),
  array([37, 38, 39]),
  array([40, 41]),
  array([40, 41]),
  array([42, 43]),
  array([42, 43]),
  array([44, 45, 46]),
  array([44, 45, 46]),
  array([44, 45, 46]),
  array([47, 48]),
  array([47, 48]),
  array([49, 50, 51]),
  array([49, 50, 51]),
  array(

In [7]:
boolean_coplanars

array([[ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False,  True,  True],
       [False, False, False, ..., False,  True,  True]])

In [8]:
# simplex_indices[:, None][boolean_coplanars]

In [9]:
nanarray = np.full_like(boolean_coplanars, np.nan, dtype=float)

In [10]:
print(np.where(boolean_coplanars, coplanarray, np.nan))

[[ 0. nan nan ... nan nan nan]
 [nan  1. nan ... nan nan nan]
 [nan nan  2. ... nan nan nan]
 ...
 [nan nan nan ... 93. nan nan]
 [nan nan nan ... nan 94. 95.]
 [nan nan nan ... nan 94. 95.]]


In [11]:
%timeit -r 10 -n 100000 np.where(boolean_coplanars, coplanarray, np.nan)

6.75 µs ± 224 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)


In [12]:
nancop = np.where(boolean_coplanars, coplanarray, -1)

nancop = np.unique(nancop, axis=0)
nancop

array([[-1, -1, -1, ..., -1, 94, 95],
       [-1, -1, -1, ..., 93, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1,  2, ..., -1, -1, -1],
       [-1,  1, -1, ..., -1, -1, -1],
       [ 0, -1, -1, ..., -1, -1, -1]])

In [13]:
# %timeit -r 10 -n 10000
# [set(nancop[i]) - {-1} for i in range(hull.nsimplex)]

In [16]:
boolean_coplanars * simplex_indices

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  1,  0, ...,  0,  0,  0],
       [ 0,  0,  2, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 93,  0,  0],
       [ 0,  0,  0, ...,  0, 94, 95],
       [ 0,  0,  0, ...,  0, 94, 95]])

In [17]:
simplex_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])

In [18]:
(simplex_indices + 1) * boolean_coplanars

array([[ 1,  0,  0, ...,  0,  0,  0],
       [ 0,  2,  0, ...,  0,  0,  0],
       [ 0,  0,  3, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 94,  0,  0],
       [ 0,  0,  0, ...,  0, 95, 96],
       [ 0,  0,  0, ...,  0, 95, 96]])

In [20]:
%%timeit -r 10 -n 1000
_, idx = np.unique(boolean_coplanars, axis=0, return_index=True)
boolean_coplanars[np.sort(idx)]

811 µs ± 58.2 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [ ]:
%%timeit -r 10 -n 1000
mbri._find_coplanar_simplices()

209 µs ± 2.23 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [21]:
cube.vertices.round(1e-15)

TypeError: 'float' object cannot be interpreted as an integer